In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import os
from sklearn.model_selection import train_test_split
import pandas as pd
from pydataset import data
from env import get_db_url
import pandas as pd
import numpy as np



## Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [2]:
def new_zillow_data():
    """
    Fetches Zillow data from the database using SQL query.

    Returns:
    - df: DataFrame containing Zillow data
    """
   
    conn = get_db_url('zillow')

    query = '''
        SELECT *
            FROM properties_2017 p
            JOIN predictions_2017 pr ON p.parcelid = pr.parcelid
            WHERE p.unitcnt = 1
            '''
    
    df = pd.read_sql(query, conn)
    return df

## Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [3]:
def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
    """
    Drops columns and rows from a DataFrame based on the percentage of missing values.
    
    Args:
        df (pandas.DataFrame): The DataFrame to handle missing values.
        prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
        prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).
        
    Returns:
        pandas.DataFrame: The DataFrame with dropped rows and columns.
    """
    
    threshold_col = int(round(prop_required_column * len(df.index)))
    df = df.dropna(thresh=threshold_col, axis=1)
    
  
    threshold_row = int(round(prop_required_row * len(df.columns)))
    df = df.dropna(thresh=threshold_row)
    
    return df


In [4]:
def missing_value_summary(df):
    """
    Returns a DataFrame summarizing the count and percentage of missing values for each attribute in the input DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame of observations and attributes.

    Returns:
        pandas.DataFrame: The summary DataFrame with count and percentage of missing values for each attribute.
    """
    missing_counts = df.isnull().sum()
    missing_percentages = (missing_counts / len(df)) * 100
    summary_df = pd.DataFrame({
        'Attribute': missing_counts.index,
        'Missing Count': missing_counts.values,
        'Missing Percentage': missing_percentages.values
    })
    return summary_df




## The input:

## A dataframe

In [5]:
df = new_zillow_data()

In [6]:
df

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,parcelid,logerror,transactiondate
0,2288172,12177905,NaN,None,None,3.0,4.0,NaN,8.0,3.0,...,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,3,12177905,-0.103410,2017-01-01
1,1970746,10887214,1.0,None,None,3.0,3.0,NaN,8.0,3.0,...,2016.0,45726.0,1533.89,None,NaN,6.037124e+13,4,10887214,0.006940,2017-01-01
2,781532,12095076,1.0,None,None,3.0,4.0,NaN,9.0,3.0,...,2016.0,496619.0,9516.26,None,NaN,6.037461e+13,6,12095076,-0.001011,2017-01-01
3,870991,12069064,NaN,None,None,1.0,2.0,NaN,5.0,1.0,...,2016.0,199662.0,2366.08,None,NaN,6.037302e+13,7,12069064,0.101723,2017-01-01
4,1246926,12790562,NaN,None,None,3.0,4.0,NaN,9.0,3.0,...,2016.0,43056.0,3104.19,None,NaN,6.037500e+13,8,12790562,-0.040966,2017-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47409,2864704,10833991,1.0,None,None,3.0,3.0,NaN,8.0,3.0,...,2016.0,114000.0,4685.34,None,NaN,6.037132e+13,77608,10833991,-0.002245,2017-09-20
47410,673515,11000655,NaN,None,None,2.0,2.0,NaN,6.0,2.0,...,2016.0,283704.0,4478.43,None,NaN,6.037101e+13,77609,11000655,0.020615,2017-09-20
47411,1843709,12773139,1.0,None,None,1.0,3.0,NaN,4.0,1.0,...,2016.0,16749.0,876.43,None,NaN,6.037434e+13,77611,12773139,0.037129,2017-09-21
47412,1187175,12826780,NaN,None,None,2.0,3.0,NaN,6.0,2.0,...,2016.0,382000.0,6317.15,None,NaN,6.037503e+13,77612,12826780,0.007204,2017-09-25


## A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).

## A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).

## The output:

## The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.

## hint:

## Look up the dropna documentation.

In [7]:
    def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
        """
        Drops columns and rows from a DataFrame based on the percentage of missing values.

        Args:
            df (pandas.DataFrame): The DataFrame to handle missing values.
            prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
            prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).

        Returns:
            pandas.DataFrame: The DataFrame with dropped rows and columns.
        """

        threshold_col = int(round(prop_required_column * len(df.index)))
        df = df.dropna(thresh=threshold_col, axis=1)


        threshold_row = int(round(prop_required_row * len(df.columns)))
        df = df.dropna(thresh=threshold_row)

        return df


## You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [8]:
df = handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5)

In [9]:
df

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,parcelid,logerror,transactiondate
0,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,3,12177905,-0.103410,2017-01-01
1,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,4,10887214,0.006940,2017-01-01
2,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,3.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,6,12095076,-0.001011,2017-01-01
3,870991,12069064,1.0,2.0,5.0,1.0,738.0,738.0,6037.0,1.0,...,18890.0,218552.0,2016.0,199662.0,2366.08,6.037302e+13,7,12069064,0.101723,2017-01-01
4,1246926,12790562,3.0,4.0,9.0,3.0,3039.0,3039.0,6037.0,3.0,...,177527.0,220583.0,2016.0,43056.0,3104.19,6.037500e+13,8,12790562,-0.040966,2017-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47409,2864704,10833991,3.0,3.0,8.0,3.0,1741.0,1741.0,6037.0,3.0,...,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,77608,10833991,-0.002245,2017-09-20
47410,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,77609,11000655,0.020615,2017-09-20
47411,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,77611,12773139,0.037129,2017-09-21
47412,1187175,12826780,2.0,3.0,6.0,2.0,1762.0,1762.0,6037.0,2.0,...,140000.0,522000.0,2016.0,382000.0,6317.15,6.037503e+13,77612,12826780,0.007204,2017-09-25


## Encapsulate your work inside of functions in a wrangle_zillow.py module.

In [10]:
def new_zillow_data():
    """
    Fetches Zillow data from the database using SQL query.

    Returns:
    - df: DataFrame containing Zillow data
    """
   
    conn = get_db_url('zillow')

    query = '''
            SELECT *
            FROM properties_2017 p
            JOIN predictions_2017 pr ON p.parcelid = pr.parcelid
            WHERE p.unitcnt = 1
            '''

    
    df = pd.read_sql(query, conn)
    return df




    def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
        """
        Drops columns and rows from a DataFrame based on the percentage of missing values.

        Args:
            df (pandas.DataFrame): The DataFrame to handle missing values.
            prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
            prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).

        Returns:
            pandas.DataFrame: The DataFrame with dropped rows and columns.
        """

        threshold_col = int(round(prop_required_column * len(df.index)))
        df = df.dropna(thresh=threshold_col, axis=1)


        threshold_row = int(round(prop_required_row * len(df.columns)))
        df = df.dropna(thresh=threshold_row)

        return df

